Title: Population Statisitics

Author: Philip Cullen

Task:

Part 1:

Write a jupyter notebook that analyses the differences between the sexes by age in Ireland:

- Weighted mean age (by sex)

- The difference between the sexes by age

Part 2:

In the same notebook, make a variable that stores an age (say 35)

- Write that code that would group the people within 5 years of that age together, into one age group 

- Calculate the population difference between the sexes in that age group.

Part 3:

- Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

# Part 1

In [52]:
import pandas as pd

In [53]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


In [54]:
df = df[df["Sex"] != "Both sexes"]
df.tail(3)

,STATISTIC,Statistic Label,TLIST(A1),CensusYear,C02199V02655,Sex,C02076V03371,Single Year of Age,C03789V04537,Administrative Counties,UNIT,VALUE
9789,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-149d-13a3-e055-000000000001,Cavan County Council,Number,12
9790,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-14a4-13a3-e055-000000000001,Donegal County Council,Number,31
9791,FY006AC01,Population,2022,2022,2,Female,650,100 years and over,2ae19629-1495-13a3-e055-000000000001,Monaghan County Council,Number,7


In [55]:
headers = df.columns.tolist()
headers

['STATISTIC',
 'Statistic Label',
 'TLIST(A1)',
 'CensusYear',
 'C02199V02655',
 'Sex',
 'C02076V03371',
 'Single Year of Age',
 'C03789V04537',
 'Administrative Counties',
 'UNIT',
 'VALUE']

In [56]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear', 'C02199V02655','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df = df[df["Single Year of Age"] != "All ages"] 
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0') 
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6464 entries, 3296 to 9791
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Sex                      6464 non-null   object
 1   Single Year of Age       6464 non-null   int64 
 2   Administrative Counties  6464 non-null   object
 3   VALUE                    6464 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 252.5+ KB


<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
/tmp/ipykernel_1808/3063851068.py:5: SyntaxWarning: invalid escape sequence '\D'
  df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)


In [57]:
df_anal = pd.pivot_table(df, 'VALUE',"Single Year of Age", "Sex")
print (df_anal.head())
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

Sex                    Female       Male
Single Year of Age                      
0                   1761.6250  1850.6250
1                   1721.5625  1804.6875
2                   1810.8750  1889.7500
3                   1842.6875  1937.5625
4                   1863.6875  1980.3750


In [58]:
headers = list(df_anal.columns)
female = headers[0]
male = headers[1]
female, male

('Female', 'Male')

In [59]:
number_female = df_anal[female].sum()
number_female

np.float64(162786.875)

In [60]:
number_male = df_anal[male].sum()
number_male

np.float64(159034.3125)

In [61]:
df_anal

Sex,Female,Male
Single Year of Age,,
0,1761.6250,1850.6250
1,1721.5625,1804.6875
2,1810.8750,1889.7500
3,1842.6875,1937.5625
4,1863.6875,1980.3750
...,...,...
96,59.7500,20.4375
97,45.7500,13.5625
98,30.7500,8.1250


### (a) Weighted Mean by Sex

In [62]:
import numpy as np
w_mean = np.average(df_anal.index, weights=df_anal[female])
w_mean

# weighted mean for females

np.float64(38.9397958987787)

In [63]:
w_mean = np.average(df_anal.index, weights=df_anal[male])
w_mean

# weighted mean for males

np.float64(37.7394477371039)

Females Weighted Mean = 38.9397958987787

Males Weighted Mean = 37.7394477371039

### (b) Difference Between Sexes by Age

In [64]:
df_anal["Difference"] = df_anal[female] - df_anal[male]
df_anal


Sex,Female,Male,Difference
Single Year of Age,,,
0,1761.6250,1850.6250,-89.0000
1,1721.5625,1804.6875,-83.1250
2,1810.8750,1889.7500,-78.8750
3,1842.6875,1937.5625,-94.8750
4,1863.6875,1980.3750,-116.6875
...,...,...,...
96,59.7500,20.4375,39.3125
97,45.7500,13.5625,32.1875
98,30.7500,8.1250,22.6250


## Part 2: Variable Storage

In this part, I select an age (35 years) and group all people within a 5-year range of that age into a single age group. 

Using this filtered group, I calculate the population difference between males and females. 

This helps show how the balance between the sexes changes for a specific age band in the dataset.

In [65]:
# Part 2 — group ages within ±5 years of a chosen age (35 years)

target_age = 35

# Compute lower and upper bounds
age_lower = target_age - 5
age_upper = target_age + 5

# Select ages in this range
age_group = df_anal.loc[age_lower:age_upper]

print("Selected Age Group:")
print(age_group)

# Sum male and female populations in this age block
total_female = age_group[female].sum()
total_male = age_group[male].sum()

# Calculate difference (Male - Female)
age_group_difference = total_male - total_female

# Output
print(f"\nPopulation difference (Male - Female) for ages {age_lower}–{age_upper}:")
print(age_group_difference)

Selected Age Group:
Sex                    Female       Male  Difference
Single Year of Age                                  
30                  2052.5625  1928.6250    123.9375
31                  2106.8750  2014.8125     92.0625
32                  2148.8750  2025.8125    123.0625
33                  2155.5625  1993.0000    162.5625
34                  2267.7500  2070.0625    197.6875
35                  2371.2500  2168.4375    202.8125
36                  2439.3750  2239.2500    200.1250
37                  2449.5625  2276.6875    172.8750
38                  2556.3750  2344.5625    211.8125
39                  2662.0000  2421.8125    240.1875
40                  2696.4375  2518.8125    177.6250

Population difference (Male - Female) for ages 30–40:
-1904.75


## Part 3: Population Difference by Region

In this part, I extend the previous analysis by identifying which region in Ireland shows the largest population difference between males and females for the selected age group. 

After filtering the dataset to include only people within 5 years of the chosen age, I group the data by region and calculate the male–female population difference for each region. 

Finally, I determine which region has the biggest difference.

In [66]:
print(df.columns.tolist())

['Sex', 'Single Year of Age', 'Administrative Counties', 'VALUE']


In [68]:
# Part 3 - Region with biggest male–female difference

age = 35

# Filter for ages within ±5 of the chosen age
df_age_group = df[
    (df["Single Year of Age"] >= age - 5) &
    (df["Single Year of Age"] <= age + 5)
]

# Group by region and sex, sum the population
region_group = df_age_group.groupby(["Administrative Counties", "Sex"])["VALUE"].sum().unstack()

# Create a column for the difference (Male - Female)
region_group["Difference"] = region_group["Male"] - region_group["Female"]

# Find the region with the largest difference
max_diff_region = region_group["Difference"].idxmax()
max_diff_value = region_group.loc[max_diff_region, "Difference"]

print("Region with the largest male–female population difference:")
print(f"{max_diff_region}: {max_diff_value}")

Region with the largest male–female population difference:
Dublin City Council: 1036


## Weighted Median by Sex (Optional)

### Weighted Median Female

In [ ]:
cumsum = df_anal[female].cumsum()
cumsum

Single Year of Age
0        1761.6250
1        3483.1875
2        5294.0625
3        7136.7500
4        9000.4375
          ...     
96     162652.8750
97     162698.6250
98     162729.3750
99     162750.3750
100    162786.8750
Name: Female, Length: 101, dtype: float64

In [ ]:
# sum of all female
cutoff = df_anal[female].sum()/2
cutoff

df_anal[female][cumsum>=cutoff].index[0]

np.int64(39)

In [ ]:
# sum of all male
cutoff = df_anal[male].sum()/2
cutoff

df_anal[male][cumsum>=cutoff].index[0]

np.int64(38)